In [ ]:
https://colab.research.google.com/drive/1QJAvIFI4ns4b4BtaPRIVSeCyxkjZhq4r?usp=sharing


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -uq "/content/drive/My Drive/faskmask/archive" -d "/content/drive/My Drive/faskmask/archive"

In [ ]:
import os
main_dir = '/content/drive/MyDrive/faskmask/archive/New Masks Dataset'
train_dir = os.path.join(main_dir,'Train')
test_dir = os.path.join(main_dir,'Test')
valid_dir = os.path.join(main_dir,'Validation')

train_mask_dir = os.path.join(train_dir,'Mask')
train_nomask_dir = os.path.join(train_dir,'Non Mask')


In [ ]:
print(train_dir)

/content/drive/MyDrive/faskmask/archive/New Masks Dataset/Train


In [ ]:
train_mask_names = os.listdir(train_mask_dir)
print(train_mask_names[:10])

train_nomask_names = os.listdir(train_nomask_dir)
print(train_nomask_names[:10])

['0022.jpg', '0040.jpg', '0003.jpg', '0037.jpg', '0006.jpg', '0029.jpg', '0018.jpg', '0042.jpg', '0019.jpg', '0027.jpg']
['1.jpg', '132.jpg', '153.jpg', '136.jpg', '139.jpg', '142.jpg', '120.jpg', '155.jpg', '129.jpg', '117.jpg']


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range = 0.2,
                                   rotation_range = 40,
                                   horizontal_flip = True
                                   )
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary'
                                                    )
test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary'
                                                    )
valid_generator = validation_datagen.flow_from_directory(valid_dir,
                                                    target_size=(150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary'
                                                    )

Found 600 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 306 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'Mask': 0, 'Non Mask': 1}

In [ ]:
train_generator.image_shape

(150, 150, 3)

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding='SAME',activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64,(3,3),padding='SAME',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 37, 37, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 87616)             0

In [ ]:
model.compile(Adam(lr=0.001),loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
history = model.fit(train_generator,
                    epochs = 30,
                    validation_data = valid_generator)

Epoch 1/30
19/19 [==============================] - 257s 12s/step - loss: 3.3672 - accuracy: 0.5400 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 2/30
19/19 [==============================] - 13s 688ms/step - loss: 0.6493 - accuracy: 0.5983 - val_loss: 0.5776 - val_accuracy: 0.8758
Epoch 3/30
19/19 [==============================] - 13s 701ms/step - loss: 0.4355 - accuracy: 0.8233 - val_loss: 0.4275 - val_accuracy: 0.8791
Epoch 4/30
19/19 [==============================] - 14s 729ms/step - loss: 0.3573 - accuracy: 0.8733 - val_loss: 0.3572 - val_accuracy: 0.8987
Epoch 5/30
19/19 [==============================] - 13s 687ms/step - loss: 0.3137 - accuracy: 0.8983 - val_loss: 0.3392 - val_accuracy: 0.9150
Epoch 6/30
19/19 [==============================] - 13s 701ms/step - loss: 0.2791 - accuracy: 0.9017 - val_loss: 0.3092 - val_accuracy: 0.9085
Epoch 7/30
19/19 [==============================] - 13s 685ms/step - loss: 0.2662 - accuracy: 0.8933 - val_loss: 0.2605 - val_accuracy: 0.9085


In [ ]:
test_loss , test_acc = model.evaluate(test_generator)
print('test acc :{} test loss:{}'.format(test_acc,test_loss))

4/4 [==============================] - 22s 7s/step - loss: 0.1607 - accuracy: 0.9400
test acc :0.9399999976158142 test loss:0.16070933640003204


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()
#print(uploaded)
for f in uploaded.keys():
  img_path='/content/'+f
  img = image.load_img(img_path , target_size=(150,150))
  images = image.img_to_array(img)
  images=np.expand_dims(images,axis=0)
  prediction = model.predict(images)
  if prediction==0:
    print(f ,'mask is there')
  else:
    print(f,'nomask is present')

Saving m1.jpg to m1 (2).jpg
Saving n1.jpg to n1.jpg
m1.jpg mask is there
n1.jpg nomask is present


In [ ]:
model.save('saved_model.h5')